In [1]:
import os
import numpy as np
import tensorflow as tf
from src.datasets import load_dataset, preprocess_dataset, prefetch_dataset
from src.models import train_model

2024-01-31 13:36:55.744076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 13:36:55.744128: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 13:36:55.744148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 13:36:55.751092: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cfg = { 'dataset' : 'svhn',
        'model' : 'densenet121',
        'batch_size' : 512,
        'optimizer' : 'Adam',
        'learning_rate' : 0.001,
        'max_epoch' : 300,
        'patience' : 10,}    

model_name = cfg['model']
dataset_name = cfg['dataset']

In [6]:
for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}'
    if not os.path.exists(exp_name):
        print("Making directory", exp_name)
        os.makedirs(exp_name)
        
    ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
    n_classes = ds_info.features['label'].num_classes
    ds_train = preprocess_dataset(ds_train, cfg, n_classes, resize=False, normalize=True, onehot=True)
    ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
    
    ##############################################################
    #
    # Train Model
    #
    # #############################################################
    
    ds_train = prefetch_dataset(ds_train, batch_size=cfg['batch_size'])
    ds_val = prefetch_dataset(ds_val, batch_size=cfg['batch_size'])
    model, history = train_model(ds_train=ds_train,
                                 ds_val=ds_val,
                                 cfg=cfg,
                                 exp_name=exp_name,
                                 save=True) 

Epoch 1/300
122/122 [==============================] - 68s 86ms/step - loss: 0.9277 - accuracy: 0.6922 - val_loss: 1.8061 - val_accuracy: 0.3727
Epoch 2/300
122/122 [==============================] - 8s 68ms/step - loss: 0.2790 - accuracy: 0.9209 - val_loss: 1.3287 - val_accuracy: 0.5597
Epoch 3/300
122/122 [==============================] - 8s 68ms/step - loss: 0.2085 - accuracy: 0.9426 - val_loss: 1.2381 - val_accuracy: 0.5880
Epoch 4/300
122/122 [==============================] - 8s 66ms/step - loss: 0.1732 - accuracy: 0.9530 - val_loss: 1.6927 - val_accuracy: 0.4722
Epoch 5/300
122/122 [==============================] - 8s 66ms/step - loss: 0.1588 - accuracy: 0.9562 - val_loss: 1.6988 - val_accuracy: 0.4970
Epoch 6/300
122/122 [==============================] - 8s 67ms/step - loss: 0.1364 - accuracy: 0.9625 - val_loss: 1.8643 - val_accuracy: 0.4390
Epoch 7/300
122/122 [==============================] - 8s 68ms/step - loss: 0.1229 - accuracy: 0.9670 - val_loss: 1.7410 - val_accuracy

In [7]:
ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
n_classes = ds_info.features['label'].num_classes
ds_train = preprocess_dataset(ds_train, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_train = prefetch_dataset(ds_train, batch_size=cfg['batch_size'])
ds_val = prefetch_dataset(ds_val, batch_size=cfg['batch_size'])
ds_test = prefetch_dataset(ds_test, batch_size=cfg['batch_size'])

##############################################################
#
# Compute the train, validation, test error
#
# #############################################################

train_acc = []
val_acc = []
test_acc = []
for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
    train_acc.append(model.evaluate(ds_train, verbose=1)[1])
    val_acc.append(model.evaluate(ds_val, verbose=1)[1])
    test_acc.append(model.evaluate(ds_test, verbose=1)[1])
print(f'Average train error: {(100-np.mean(train_acc)*100):.2f}, std: {(np.std(train_acc)*100):.2f}')
print(f'Average validation error: {(100-np.mean(val_acc)*100):.2f}, std: {(np.std(val_acc)*100):.2f}')
print(f'Average test error: {(100-np.mean(test_acc)*100):.2f}, std: {(np.std(test_acc)*100):.2f}')

51/51 [==============================] - 1s 12ms/step - loss: 0.4319 - accuracy: 0.9234
Average train error: 1.42, std: 1.10
Average validation error: 6.66, std: 0.61
Average test error: 6.35, std: 0.67
